In [1]:
!pip install -U keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_minst = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_minst.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
# Normalization( pixel values in images are usually in the range [0, 255], 0:Black,255:White)
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [16]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters= hp.Int('conv_1_filter',min_value= 64,max_value=128, step=16),
          kernel_size = hp.Choice('conv_1_kernal',values = [3,5]),
          activation= 'relu',
          input_shape=(28,28,1)

      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
          kernel_size = hp.Choice('conv_2_kernal', values = [3,5]),
          activation= 'relu'

      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units',min_value=32, max_value=128, step= 16),
          activation= 'relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(
      optimizer= keras.optimizers.Adam(
          hp.Choice('learning_rate',values= [1e-2, 1e-3])
      ),
      loss = 'sparse_categorical_crossentropy',
      metrics= ['accuracy']
  )
  return model

In [17]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search= RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory= 'output',project_name='mnistfashion')

In [20]:
tuner_search.search(train_images,train_labels,epochs= 3, validation_split= 0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.8688333630561829

Best val_accuracy So Far: 0.9075000286102295
Total elapsed time: 00h 10m 14s


In [21]:
#To get Top model
model = tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 96)        2496      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        55360     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 128)               3965056   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 4024202 (15.35 MB)
Trainable params: 4024202 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 12s 6ms/step - loss: 0.1484 - accuracy: 0.9434 - val_loss: 0.2663 - val_accuracy: 0.9107
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1090 - accuracy: 0.9593 - val_loss: 0.2957 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0822 - accuracy: 0.9701 - val_loss: 0.3168 - val_accuracy: 0.9145
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0568 - accuracy: 0.9790 - val_loss: 0.3297 - val_accuracy: 0.9158
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0455 - accuracy: 0.9830 - val_loss: 0.3619 - val_accuracy: 0.9140
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0370 - accuracy: 0.9869 - val_loss: 0.3998 - val_accuracy: 0.9172
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0300 - accuracy: 0.9896 - val_loss: 0.4852 - val_a